In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 텍스트 지정하여 가져오기

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("user-agent=Mozilla/5.0 ...")  # 그대로 유지
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

# 소속사 및 연예인 정보 (예시)
artist_data = [
    {'agency': 'YG', 'artist': '제니', 'url': 'https://namu.wiki/w/BLACKPINK/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0#%EC%A0%9C%EB%8B%88'},
    # 추가...
]

with open('제니_크롤링21.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['소속사', '연예인', '날짜', '사건 제목', '사건 내용', '대응 방식'])

    for artist in artist_data:
        driver.get(artist['url'])
        time.sleep(2)

        # 목차 span 태그 중 번호 있는 항목만 필터링
        toc_elements = driver.find_elements(By.CSS_SELECTOR, 'span._79wNnvjA')
        for elem in toc_elements:
            try:
                text = elem.text.strip()
                if re.match(r'\d+\.\d+', text):  # 예: 2.1, 2.2, 3.1.1 ...
                    title = elem.find_element(By.TAG_NAME, 'a').text
                    anchor = elem.find_element(By.TAG_NAME, 'a').get_attribute('href').split('#')[-1]

                    # 자바스크립트로 해당 위치로 이동
                    driver.execute_script(f"location.href='#{anchor}'")
                    time.sleep(1)

                    # 이동 후 해당 섹션 가져오기 (정확한 구조는 테스트 필요)
                    section_elem = driver.find_element(By.ID, anchor)
                    parent_elem = section_elem.find_element(By.XPATH, './../..')  # 상위 div 접근
                    content_text = parent_elem.text.strip()

                    # 날짜 정규식 추출 (여러 날짜 중 첫 번째만)
                    date_match = re.search(r'\d{4}년\s?\d{1,2}월\s?\d{1,2}일', content_text)
                    date = date_match.group() if date_match else ''

                    writer.writerow([artist['agency'], artist['artist'], date, title, content_text])
            except Exception as e:
                print(f"오류 발생: {e}")
                continue

driver.quit()

## 크롤링은 되는데 사건제목에 나무위키 제목 숫자가, 내용에 사건 제목이 들어가고 있음

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re
import csv
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("user-agent=Mozilla/5.0 ...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

# 소속사 및 연예인 정보
artist_data = [
    {'agency': 'YG', 'artist': '제니', 'url': 'https://namu.wiki/w/BLACKPINK/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0#%EC%A0%9C%EB%8B%88'},
]

with open('제니_크롤링5.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['소속사', '연예인', '날짜', '사건 제목', '사건 내용', '대응 방식'])

    for artist in artist_data:
        driver.get(artist['url'])
        time.sleep(2)

        toc_elements = driver.find_elements(By.CSS_SELECTOR, 'span._79wNnvjA')
        for elem in toc_elements:
            try:
                text = elem.text.strip()
                if re.match(r'\d+\.\d+', text):  # 예: 2.1, 2.2 ...
                    anchor = elem.find_element(By.TAG_NAME, 'a').get_attribute('href').split('#')[-1]

                    # 해당 위치로 이동
                    driver.execute_script(f"location.href='#{anchor}'")
                    time.sleep(1)

                    # 해당 ID 요소 기준으로 부모 div에서 텍스트 가져옴
                    section_elem = driver.find_element(By.ID, anchor)
                    parent_elem = section_elem.find_element(By.XPATH, './../..')
                    actual_title = parent_elem.text.strip()  # ← 이게 사건 제목

                    # 날짜는 아직 사용하지 않음
                    writer.writerow([artist['agency'], artist['artist'], '', actual_title, '', ''])
            except Exception as e:
                print(f"오류 발생: {e}")
                continue

driver.quit()

## 사건제목까지는는 잘됨

### 전체 텍스트 가져오는 방법

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# 크롬 드라이버 설정
options = Options()
# options.add_argument("--headless=new")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--lang=ko_KR")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# URL 열기
url = 'https://namu.wiki/w/BLACKPINK/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0'
driver.get(url)
time.sleep(3)

# 페이지 전체 HTML 소스 가져오기
html = driver.page_source

# BeautifulSoup으로 파싱
soup = BeautifulSoup(html, 'html.parser')

# 본문 텍스트 전체 추출
text = soup.get_text(separator="\n", strip=True)

# 확인용 출력 (혹은 파일 저장도 가능)
print(text)

# 파일로 저장하고 싶다면:
with open("나무위키_전체본문.txt", "w", encoding="utf-8") as f:
    f.write(text)

driver.quit()


BLACKPINK/논란 및 사건 사고 - 나무위키
최근 변경
최근 토론
특수 기능
12
편집
편집
편집 권한이 부족합니다. 로그인된 사용자(이)여야 합니다. 해당 문서의
ACL 탭
을 확인하시기 바랍니다.
닫기
토론
역사
BLACKPINK/논란 및 사건 사고
최근 수정 시각:
2025-03-26 01:17:59
분류
BLACKPINK
대한민국의 인물별 논란
대한민국의 아이돌 사건 사고
상위 문서:
BLACKPINK
주의. 사건·사고 관련 내용을 설명합니다.
사건 사고 관련 서술 규정
을 유의하시기 바랍니다.
지수
제니
로제
리사
[ 한국 음반 ]
SQUARE ONE
디지털 싱글
2016. 08. 08.
SQUARE TWO
디지털 싱글
2016. 11. 01.
마지막처럼
디지털 싱글
2017. 06. 22.
SQUARE UP
미니 1집
2018. 06. 15.
KILL THIS LOVE
미니 2집
2019. 04. 05.
How You Like That
선공개 싱글
2020. 06. 26.
Ice Cream
선공개 싱글
2020. 08. 28.
THE ALBUM
정규 1집
2020. 10. 02.
Pink Venom
선공개 싱글
2022. 08. 19.
BORN PINK
정규 2집
2022. 09. 16.
미정
미정
2025. XX. XX.
[ 일본 음반 ]
BLACKPINK
미니 1집
2017. 08. 30.
Re: BLACKPINK
리패키지
2018. 03. 28.
DDU-DU DDU-DU
싱글 1집
2018. 08. 22.
BLACKPINK
IN YOUR AREA
정규 1집
2018. 12. 05.
Kill This Love (JP Ver.)
미니 2집
2019. 10. 16.
THE ALBUM -JP Ver.-
정규 2집
2021. 08. 03.
[ 참여 음반 ]
Kiss and Make Up
2018. 10. 19.
Sour Candy
2020. 05. 28.
Ready For Love
2022. 07. 29.
THE GIRLS
2023. 08. 25.

### 파일로 저장 형태 확인

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv

# 크롬 드라이버 설정
options = Options()
# options.add_argument("--headless=new")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--lang=ko_KR")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# ✅ 연예인 정보 딕셔너리 리스트
artist_data = [
    {
        '소속사': 'YG',
        '그룹': 'BLACKPINK',
        '연예인 이름': '제니',
        'url': 'https://namu.wiki/w/BLACKPINK/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0#%EC%A0%9C%EB%8B%88'
    },
    # 필요하면 여기에 추가
]

# ✅ CSV 저장
with open("연예인_전체텍스트.csv", "w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(['소속사', '그룹', '연예인 이름', '텍스트'])  # 헤더

    for artist in artist_data:
        driver.get(artist['url'])
        time.sleep(3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 전체 텍스트 추출
        text = soup.get_text(separator="\n", strip=True)

        # CSV에 쓰기
        writer.writerow([
            artist['소속사'],
            artist['그룹'],
            artist['연예인 이름'],
            text
        ])

driver.quit()


### 연예인 나무위키 사이트까지 지정

In [24]:
import pandas as pd

# 연예인 목록 데이터 (원하는 만큼 추가 가능)
data = [
    # 빅뱅
    {
        '소속사': 'YG',
        '그룹': '빅뱅',
        '연예인 이름': '승리',
        'url': 'https://namu.wiki/w/%EC%8A%B9%EB%A6%AC(%EC%9D%B8%EB%AC%BC)/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0'
    },
    {
        '소속사': 'YG',
        '그룹': '빅뱅',
        '연예인 이름': '탑',
        'url': 'https://namu.wiki/w/T.O.P/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0'
    },
    {
        '소속사': 'YG',
        '그룹': '빅뱅',
        '연예인 이름': '태양',
        'url': ''
    },
    {
        '소속사': 'YG',
        '그룹': '빅뱅',
        '연예인 이름': '지드래곤',
        'url': 'https://namu.wiki/w/G-DRAGON/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0'
    },
    {
        '소속사': 'YG',
        '그룹': '빅뱅',
        '연예인 이름': '대성',
        'url': 'https://namu.wiki/w/%EB%8C%80%EC%84%B1(BIGBANG)/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0'
    },
    # 블랙핑크
    {
        '소속사': 'YG',
        '그룹': '블랙핑크',
        '연예인 이름': '제니',
        'url': 'https://namu.wiki/w/BLACKPINK/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0#%EC%A0%9C%EB%8B%88'
    },
    {
        '소속사': 'YG',
        '그룹': '블랙핑크',
        '연예인 이름': '지수',
        'url': ''
    },
    {
        '소속사': 'YG',
        '그룹': '블랙핑크',
        '연예인 이름': '리사',
        'url': 'https://namu.wiki/w/BLACKPINK/%EB%85%BC%EB%9E%80%20%EB%B0%8F%20%EC%82%AC%EA%B1%B4%20%EC%82%AC%EA%B3%A0#%EB%A6%AC%EC%82%AC'
    },
    {
        '소속사': 'YG',
        '그룹': '블랙핑크',
        '연예인 이름': '로제',
        'url': ''
    }
]

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("YG_연예인목록.csv", index=False, encoding='utf-8-sig')

### 연예인 목록 사용하여 전체 텍스트 가져오기

In [26]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv

# 크롬 드라이버 설정
options = Options()
# options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--lang=ko_KR")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# 연예인 목록 CSV 불러오기
artist_df = pd.read_csv("YG_연예인목록.csv")

# 결과 저장 CSV
with open("YG_연예인_전체텍스트.csv", "w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(['소속사', '그룹', '연예인 이름', '텍스트'])

    for _, artist in artist_df.iterrows():
        url = str(artist['url']).strip()
        
        if not url.startswith("http"):
            # URL이 없으면 텍스트 없이 기본 정보만 기록
            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                ''
            ])
            print(f"ℹ️ URL 없음 → 텍스트 없이 기록: {artist['연예인 이름']}")
            continue

        try:
            driver.get(url)
            time.sleep(3)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            text = soup.get_text(separator="\n", strip=True)

            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                text
            ])
        except Exception as e:
            print(f"❌ 크롤링 오류: {artist['연예인 이름']} → {e}")
            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                ''
            ])

driver.quit()

ℹ️ URL 없음 → 텍스트 없이 기록: 태양
ℹ️ URL 없음 → 텍스트 없이 기록: 지수
ℹ️ URL 없음 → 텍스트 없이 기록: 로제


In [30]:
df = pd.read_csv("YG_연예인_전체텍스트.csv")
artist_text = df[df['연예인 이름'] == '제니']['텍스트'].values

print(artist_text[0])

BLACKPINK/논란 및 사건 사고 - 나무위키
최근 변경
최근 토론
특수 기능
12
편집
편집
편집 권한이 부족합니다. 로그인된 사용자(이)여야 합니다. 해당 문서의
ACL 탭
을 확인하시기 바랍니다.
닫기
토론
역사
BLACKPINK/논란 및 사건 사고
최근 수정 시각:
2025-03-26 01:17:59
분류
BLACKPINK
대한민국의 인물별 논란
대한민국의 아이돌 사건 사고
상위 문서:
BLACKPINK
주의. 사건·사고 관련 내용을 설명합니다.
사건 사고 관련 서술 규정
을 유의하시기 바랍니다.
지수
제니
로제
리사
[ 한국 음반 ]
SQUARE ONE
디지털 싱글
2016. 08. 08.
SQUARE TWO
디지털 싱글
2016. 11. 01.
마지막처럼
디지털 싱글
2017. 06. 22.
SQUARE UP
미니 1집
2018. 06. 15.
KILL THIS LOVE
미니 2집
2019. 04. 05.
How You Like That
선공개 싱글
2020. 06. 26.
Ice Cream
선공개 싱글
2020. 08. 28.
THE ALBUM
정규 1집
2020. 10. 02.
Pink Venom
선공개 싱글
2022. 08. 19.
BORN PINK
정규 2집
2022. 09. 16.
미정
미정
2025. XX. XX.
[ 일본 음반 ]
BLACKPINK
미니 1집
2017. 08. 30.
Re: BLACKPINK
리패키지
2018. 03. 28.
DDU-DU DDU-DU
싱글 1집
2018. 08. 22.
BLACKPINK
IN YOUR AREA
정규 1집
2018. 12. 05.
Kill This Love (JP Ver.)
미니 2집
2019. 10. 16.
THE ALBUM -JP Ver.-
정규 2집
2021. 08. 03.
[ 참여 음반 ]
Kiss and Make Up
2018. 10. 19.
Sour Candy
2020. 05. 28.
Ready For Love
2022. 07. 29.
THE GIRLS
2023. 08. 25.

### div 지정해서 텍스트 가져올 수 있는지

In [31]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv

# 크롬 드라이버 설정
options = Options()
# options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--lang=ko_KR")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# 연예인 목록 CSV 불러오기
artist_df = pd.read_csv("YG_연예인목록.csv")

# 결과 저장 CSV
with open("2YG_연예인_전체텍스트.csv", "w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(['소속사', '그룹', '연예인 이름', '텍스트'])

    for _, artist in artist_df.iterrows():
        url = str(artist['url']).strip()
        
        if not url.startswith("http"):
            # URL이 없으면 텍스트 없이 기본 정보만 기록
            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                ''
            ])
            print(f"ℹ️ URL 없음 → 텍스트 없이 기록: {artist['연예인 이름']}")
            continue

        try:
            driver.get(url)
            time.sleep(3)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # BEFORE: 페이지 전체 텍스트
            # text = soup.get_text(separator="\n", strip=True)

            # AFTER: 특정 div 텍스트만
            target_div = soup.find("div", class_="SwHvVAMU")
            text = target_div.get_text(separator="\n", strip=True) if target_div else ""

            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                text
            ])
        except Exception as e:
            print(f"❌ 크롤링 오류: {artist['연예인 이름']} → {e}")
            writer.writerow([
                artist['소속사'],
                artist['그룹'],
                artist['연예인 이름'],
                ''
            ])

driver.quit()

ℹ️ URL 없음 → 텍스트 없이 기록: 태양
ℹ️ URL 없음 → 텍스트 없이 기록: 지수
ℹ️ URL 없음 → 텍스트 없이 기록: 로제


In [32]:
df = pd.read_csv("YG_연예인_전체텍스트.csv")
artist_text = df[df['연예인 이름'] == '제니']['텍스트'].values

print(artist_text[0])

BLACKPINK/논란 및 사건 사고 - 나무위키
최근 변경
최근 토론
특수 기능
12
편집
편집
편집 권한이 부족합니다. 로그인된 사용자(이)여야 합니다. 해당 문서의
ACL 탭
을 확인하시기 바랍니다.
닫기
토론
역사
BLACKPINK/논란 및 사건 사고
최근 수정 시각:
2025-03-26 01:17:59
분류
BLACKPINK
대한민국의 인물별 논란
대한민국의 아이돌 사건 사고
상위 문서:
BLACKPINK
주의. 사건·사고 관련 내용을 설명합니다.
사건 사고 관련 서술 규정
을 유의하시기 바랍니다.
지수
제니
로제
리사
[ 한국 음반 ]
SQUARE ONE
디지털 싱글
2016. 08. 08.
SQUARE TWO
디지털 싱글
2016. 11. 01.
마지막처럼
디지털 싱글
2017. 06. 22.
SQUARE UP
미니 1집
2018. 06. 15.
KILL THIS LOVE
미니 2집
2019. 04. 05.
How You Like That
선공개 싱글
2020. 06. 26.
Ice Cream
선공개 싱글
2020. 08. 28.
THE ALBUM
정규 1집
2020. 10. 02.
Pink Venom
선공개 싱글
2022. 08. 19.
BORN PINK
정규 2집
2022. 09. 16.
미정
미정
2025. XX. XX.
[ 일본 음반 ]
BLACKPINK
미니 1집
2017. 08. 30.
Re: BLACKPINK
리패키지
2018. 03. 28.
DDU-DU DDU-DU
싱글 1집
2018. 08. 22.
BLACKPINK
IN YOUR AREA
정규 1집
2018. 12. 05.
Kill This Love (JP Ver.)
미니 2집
2019. 10. 16.
THE ALBUM -JP Ver.-
정규 2집
2021. 08. 03.
[ 참여 음반 ]
Kiss and Make Up
2018. 10. 19.
Sour Candy
2020. 05. 28.
Ready For Love
2022. 07. 29.
THE GIRLS
2023. 08. 25.

### 구역 지정 vs all 비교

In [33]:
import pandas as pd

# 파일 불러오기
df_all = pd.read_csv("YG_연예인_전체텍스트.csv")
df_partial = pd.read_csv("2YG_연예인_전체텍스트.csv")

# 연예인 이름으로 비교할 대상 지정
target_name = '제니'

# 전체 텍스트에서 해당 연예인 텍스트 길이
text_all = df_all[df_all['연예인 이름'] == target_name]['텍스트'].fillna('').values[0]
length_all = len(text_all)

# 구역 한정 텍스트에서 해당 연예인 텍스트 길이
text_partial = df_partial[df_partial['연예인 이름'] == target_name]['텍스트'].fillna('').values[0]
length_partial = len(text_partial)

# 결과 출력
print(f"[{target_name}] 텍스트 길이 비교:")
print(f"전체 텍스트 길이: {length_all}자")
print(f"구역 한정 텍스트 길이: {length_partial}자")


[제니] 텍스트 길이 비교:
전체 텍스트 길이: 6932자
구역 한정 텍스트 길이: 6147자
